파이토치로 구현한 퍼셉트론
- 임의 개수의 입력을 받아 affine transform을 수행하고, activation function을 적용한 후 출력 하나를 만든다.

In [1]:
import torch
import torch.nn as nn

class Perceptron(nn.Module):
    """퍼셉트론은 하나의 선형 층입니다."""
    def __init__(self, input_dim):
        """
        매개변수:
            input_dim (int): 입력 특성의 크기
        """
        super(Perceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1)
    
    def forward(self, x_in):
        """
        퍼셉트론의 정방향 계산

        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, num_feautres)이다.
        반환값:
            결과 텐서. tensor.shape는 (batch, )이다.
        """
        return torch.sigmoid(self.fc1(x_in)).squeeze()

손실 함수

In [2]:
import torch
import torch.nn as nn
mse_loss = nn.MSELoss()
outputs = torch.randn(3, 5, requires_grad=True)
targets = torch.randn(3, 5)
loss = mse_loss(outputs, targets)
print(loss)

tensor(3.0891, grad_fn=<MseLossBackward0>)


크로스 엔트로피 손실

In [3]:
import torch
import torch.nn as nn

ce_loss = nn.CrossEntropyLoss()
outputs = torch.randn(3, 5, requires_grad = True)
targets = torch.tensor([1, 0, 3], dtype=torch.int64)
loss = ce_loss(outputs, targets)
print(loss)

tensor(2.3809, grad_fn=<NllLossBackward0>)


이진 크로스 엔트로피 손실

In [4]:
bce_loss = nn.BCELoss()
sigmoid = nn.Sigmoid()
probabilities = sigmoid(torch.randn(4, 1, requires_grad = True))
targets = torch.tensor([1, 0, 1, 0], dtype=torch.float32).view(4, 1)
loss = bce_loss(probabilities, targets)
print(probabilities)
print(targets)

tensor([[0.6616],
        [0.0938],
        [0.5502],
        [0.5213]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [0.],
        [1.],
        [0.]])


퍼셉트론과 이진 분류를 위한 지도 학습 훈련 반복
```
# 각 에포크는 전체 훈련 데이터를 사용한다.
for epoch_i in range(n_epochs):
    # 내부 반복은 데이터셋에 있는 배치에 대해 수행된다.
    for batch_i in range(n_batches):
        # 0단계 : 데이터 가져오기
        x_data, y_target = get_toy_data(batch_size)

        # 1단계 : 그레디언트 초기화
        perceptron.zero_grad()

        # 2단계 : 모델의 정방향 계산 수행하기
        y_pred = perceptron(x_data, apply_sigmoid = True)
        
        # 3단계 : 최적하려는 손실 계산하기
        loss = bce_loss(y_pred, y_target)

        # 4단계 : 손실 신호를 거꾸로 전파하기
        loss.backward()

        # 5단계 : 옵티마이저로 업데이트하기
        optimizer.step()
```

옐프 리뷰 데이터셋

In [5]:
import collections
import numpy as np
import pandas as pd
import re
from argparse import Namespace

In [6]:
args = Namespace(
    raw_train_dataset_csv = "../../Dataset/yelp/raw_train.csv",
    raw_test_dataset_csv = "../../Dataset/yelp/raw_test.csv",
    proportion_subset_of_train=0.1,
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv="../../Dataset/yelp/reviews_with_splits_lite.csv",
    seed=1337
)

In [7]:
# 원본 데이터 읽기
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=["rating", "review"])

In [8]:
# 리뷰 클래스 비율이 동일하도록 만든다.
by_rating = collections.defaultdict(list)
for _, row in train_reviews.iterrows():
    by_rating[row.rating].append(row.to_dict())

review_subset = []

for _, item_list in sorted(by_rating.items()):
    n_total = len(item_list)
    n_subset = int(args.proportion_subset_of_train * n_total)
    review_subset.extend(item_list[:n_subset])

review_subset = pd.DataFrame(review_subset)

In [9]:
review_subset.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,1,I don't know what Dr. Goldberg was like before...
2,1,I'm writing this review to give you a heads up...
3,1,Wing sauce is like water. Pretty much a lot of...
4,1,Owning a driving range inside the city limits ...


In [10]:
train_reviews.rating.value_counts()

rating
1    280000
2    280000
Name: count, dtype: int64

In [11]:
# 고유 클래스
set(review_subset.rating)

{1, 2}

In [12]:
# 훈련, 검증 테스트 세트를 만들기 위한 별점을 기준으로 나눈다.
by_rating = collections.defaultdict(list)
for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())

# 분할 데이터를 만든다.
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_rating.items()):
    np.random.shuffle(item_list)

    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    n_test = int(args.test_proportion * n_total)

    # 데이터 포인트에 분할 속성을 추가한다.
    for item in item_list[:n_train]:
        item["split"] = "train"

    for item in item_list[n_train:n_train + n_val]:
        item["split"] = "val"

    for item in item_list[n_train + n_val : n_train + n_val + n_test]:
        item["split"] = "test"
    
    # 최종 리스트에 추가한다.
        final_list.extend(item_list)

In [13]:
# 분할 데이터를 데이터 프레임으로 만든다.
final_reviews = pd.DataFrame(final_list)

In [14]:
final_reviews.split.value_counts()

split
train    164640000
val       35280000
test      35280000
Name: count, dtype: int64

In [15]:
# Review 전처리
def preprocessing_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

final_reviews.review = final_reviews.review.apply(preprocessing_text)

In [ ]:
final_reviews["rating"] = final_reviews.rating.apply({1:"negative", 2:"positive"}.get)

final_reviews.head()

데이터 벡터화 클래스

In [ ]:
# 옐프 리뷰 데이터를 위한 파이토치 데이터셋 클래스
from torch.utils.data import Dataset

class ReviewDataset(Dataset):
    def __init__(self, review_df, vectorizer):
        """
        매개변수:
            review_df (pandas.DataFrame): 데이터셋
            vectorizer (ReviewVectorizer): ReviewVectorizer 객체
        """
        self.review_df = review_df
        self._vectorizer = vectorizer

        self.train_df = self.review_df[self.review_df.split=="train"]
        self.train_size = len(self.train_df)

        self.val_df = self.review_df[self.review_df.split=="val"]
        self.validation_size = len(self.val_df)

        self.test_df = self.review_df[self.review_df.split=="test"]
        self.test_size = len(self.test_df)

        self._lookup_dict = {
            "train": (self.train_df, self.train_size),
            "val": (self.val_df, self.validation_size),
            "test": (self.test_df, self.test_size)
        }

        self.set_split("train")

        @classmethod
        def load_dataset_and_make_vectorizer(cls, review_csv):
            """
            데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만든다.

            매개변수:
                review_csv (str): 데이터셋의 위치
            반환값:
                ReviewDataset의 인스턴스
            """
            review_df = pd.read_csv(review_csv)
            train_review_df = review_df[review_df.split=="train"]
            return cls(review_df, ReviewVectorizer.from_dataframe(train_review_df))
        
        @classmethod
        def load_dataset_and_load_vectorizer(cls, review_csv, vectorizer_filepath):
            """
            데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만든다.
            캐시된 ReviewVectorizer 객체를 재사용할 때 사용한다.

            매개변수:
                review_csv (str): 데이터셋의 위치
                vecotrizer_filepath (str): ReviewVectorizer 객체의 저장 위치
            반환값: 
                ReviewDataset의 인스턴스
            """
            review_df = pd.read_csv(review_csv)
            vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
            return cls(review_df, vectorizer)
        
        @staticmethod
        def load_vectorizer_only(vectorizer_filepath):
            """
            파일에서 ReviceVectorizer 객체를 로드하는 정적 메서드

            매개변수:
                vectorizer_filepath (str): 직렬화된 ReviewVectorizer 객체의 위치
            반환값: 
                ReviewVectorizer의 인스턴스
            """
            with open(vectorizer_filepath) as fp:
                return ReviewVectorizer.from_serializable(json.load(fp))
            
        def save_vectorizer(self, vectorizer_filepath):
            """
            ReviewVectorizer 객체를 json 형태로 디스크에 저장한다.
            매개변수 :
                vectorizer_filepath (str): ReviewVectorizer 객체의 저장 위치
            """
            with open(vectorizer_filepath, "w") as fp:
                json.dump(self._vectorizer.to_serializable(), fp)
        
        def get_vectorizer(self):
            """
            벡터 변환 객체를 반환한다.
            """
            return self._vectorizer
        
        def set_split(self, split="train"):
            """
            데이터프레임에 있는 열을 사용하여 분할 세트를 선택한다.

            매개변수: 
                split (str): "train", "val", "test" 중 하나
            """
            self._traget_split = split
            self._target_df, self._target_size = self._lookup_dict[split]

        def __len__(self):
            return self.target_size
        
        def __getitem__(self, index):
            """
            파이토치 데이터셋의 주요 진입 메서드

            매개변수: 
                index (int): 데이터 포인트의 인덱스
            반환값: 
                데이터 포인트의 특성(x_data)과 레이블(y_target)로 이뤄진 딕셔너리
            """
            row = self._target_df.iloc[index]

            review_vector = self._vectorizer.vectorize(row.review)

            rating_index = self._vectorizer.rating_vocab.lookup_token(row.rating)

            return {
                "x_data": review_vector,
                "y_target": rating_index
            }
        
        def get_num_batches(self, batch_size):
            """
            배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수를 반환한다.

            매개변수: 
                batch_size (int)
            반환값
                배치 개수
            """
            return len(self) // batch_size

Vocabulary